In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor
from smote import smote
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
device = torch.device("cuda")
print(f"Current device: {torch.cuda.get_device_name(torch.cuda.current_device())}" if torch.cuda.is_available() else "Current device: CPU")
torch.cuda.empty_cache()

Current device: CPU


In [2]:
args = Args() 
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('data.pt')
print(data)

HeteroData(
  a={
    num_nodes=28645,
    y=[28645]
  },
  p={ num_nodes=21044 },
  v={ num_nodes=18 },
  p_title_embed={ x=[21044, 128] },
  p_abstract_embed={ x=[21044, 128] },
  p_net_embed={ x=[21044, 128] },
  p_a_net_embed={ x=[21044, 128] },
  p_p_net_embed={ x=[21044, 128] },
  p_v_net_embed={ x=[21044, 128] },
  a_net_embed={ x=[28645, 128] },
  a_text_embed={ x=[28645, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 198912] },
  (a, walk, p)={ edge_index=[2, 144965] },
  (a, walk, v)={ edge_index=[2, 42500] },
  (p, walk, a)={ edge_index=[2, 137499] },
  (p, walk, p)={ edge_index=[2, 158408] },
  (p, walk, v)={ edge_index=[2, 29341] },
  (v, walk, a)={ edge_index=[2, 180] },
  (v, walk, p)={ edge_index=[2, 180] },
  (v, walk, v)={ edge_index=[2, 31] }
)


In [3]:
print(data['a','walk','p'].edge_index)

tensor([[    0,     0,     0,  ..., 28644, 28644, 28644],
        [11846, 11765, 12754,  ...,  4378, 20250,  6612]])


In [4]:
lr = 0.0001
num_epochs = 100
weight_decay = 5e-4
embed_dim = 128
dropout = 0.1
im_class_num = [1]
im_ratio = [0.5]
class_sample_num = 200
nclass = 4
node_dim = [args.A_n, args.P_n, args.V_n]

In [5]:
c_train_num = dl.train_num(data['a'].y, im_class_num, class_sample_num, im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[200, 100, 200, 200] 700
0 3508
1 1946
2 2890
3 3016
train_idx:  [21569, 2741, 6174, 12435, 12944, 8154, 632, 26879, 28247, 11345, 18808, 20332, 22710, 452, 20759, 8412, 3257, 8539, 704, 10784, 24277, 8812, 24963, 9127, 1774, 18137, 23844, 7278, 20715, 5671, 15297, 16221, 4353, 22773, 19243, 5710, 5853, 1406, 4334, 12342, 5594, 12054, 17175, 27624, 3578, 4177, 22116, 11133, 12288, 16870, 12158, 18384, 23401, 23981, 7441, 7825, 23528, 17714, 21992, 26063, 14171, 4394, 21036, 2649, 8077, 27828, 23788, 7042, 27753, 10197, 4889, 21228, 8057, 18261, 22922, 9424, 9791, 11746, 9535, 7202, 6608, 13592, 21399, 18999, 8429, 14794, 8591, 16813, 25293, 2028, 10414, 20714, 22384, 1704, 24313, 18695, 11291, 752, 24878, 21978, 11040, 19805, 24329, 20593, 15473, 2513, 2419, 15884, 14619, 3681, 27792, 9190, 10347, 21944, 21445, 24612, 17002, 6918, 17061, 2731, 11610, 4523, 5297, 19006, 18360, 2548, 3797, 1226, 12974, 17336, 24946, 16109, 7659, 8106, 12194, 16338, 26994, 10112, 10312, 23422, 16521, 8203

In [6]:
encoder = Het_En(node_dim, embed_dim, dropout)
decoder = EdgePredictor(embed_dim)
classifier = Classifier(embed_dim, nclass, dropout)
#print(features.shape)

In [7]:
# new_features, new_labels , new_train_idx = smote(features = data['a_net_embed'].x, labels = data['a'].y, 
#                                             train_idx = train_idx, portion = 0, im_class_num = im_class_num)
# print(new_features.shape, new_labels.shape, new_train_idx.shape)

## Training Part

In [8]:
torch.cuda.empty_cache()
train_smote(data, encoder, classifier, decoder, num_epochs, lr, weight_decay, train_idx, val_idx, portion = 0, im_class_num = im_class_num, mode = 'nsm')

Epoch [1/100], Loss: 1.3874, Accuracy: 0.2343, Edge Accuracy: 1.0000
tensor([[0.2527, 0.2542, 0.2552, 0.2378],
        [0.2483, 0.2462, 0.2665, 0.2390],
        [0.2549, 0.2490, 0.2570, 0.2392],
        ...,
        [0.2476, 0.2466, 0.2640, 0.2418],
        [0.2480, 0.2536, 0.2558, 0.2426],
        [0.2528, 0.2571, 0.2511, 0.2390]], grad_fn=<SoftmaxBackward0>)
Class 0:AUC-ROC- 0.6675, F1 Score- 0.5045; Class 1:AUC-ROC- 0.4754, F1 Score- 0.2194; Class 2:AUC-ROC- 0.3656, F1 Score- 0.4385; Class 3:AUC-ROC- 0.5011, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5024,Macro-Average F1 Score: 0.2906
Validation Loss: 1.3843, Validation Accuracy: 0.2800, Validation Edge Accuracy: 1.0000
tensor([[0.2543, 0.2512, 0.2586, 0.2359],
        [0.2482, 0.2519, 0.2585, 0.2414],
        [0.2537, 0.2501, 0.2573, 0.2389],
        ...,
        [0.2542, 0.2500, 0.2567, 0.2391],
        [0.2525, 0.2519, 0.2559, 0.2397],
        [0.2489, 0.2508, 0.2588, 0.2414]])
Class 0:AUC-ROC- 0.7295, F1 Score- 0.2589; Class 1:

In [9]:
test_smote(data, encoder, classifier, decoder, test_idx, adj_old = None, mode = 'nsm', dataset = "Test")

Test Loss: 1.0106, Test Accuracy: 0.9864, Test Edge Accuracy: 1.0000
tensor([[0.3950, 0.1946, 0.2271, 0.1833],
        [0.2048, 0.2076, 0.2307, 0.3569],
        [0.3950, 0.1974, 0.2291, 0.1784],
        ...,
        [0.4000, 0.1933, 0.2263, 0.1804],
        [0.3969, 0.1911, 0.2304, 0.1816],
        [0.2023, 0.2047, 0.2365, 0.3565]])
Class 0:AUC-ROC- 0.9818, F1 Score- 0.5207; Class 1:AUC-ROC- 0.9863, F1 Score- 0.6521; Class 2:AUC-ROC- 0.9651, F1 Score- 0.2659; Class 3:AUC-ROC- 0.9798, F1 Score- 0.4942; Macro-Average AUC-ROC: 0.9782,Macro-Average F1 Score: 0.4832
